In [34]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import statsmodels.api as sm
import pandas as pd
import plotly.express as px
from scipy.signal import savgol_filter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from linearmodels.panel import PanelOLS




In [24]:
# categorical data 

cat_data = pd.read_csv('data/bank_comps.csv')

# create 2 groups based on the number of employees, assets

# Calculate median values
median_employee = cat_data['number_of_employee'].median()
median_assets = cat_data['assets_under_management'].median()

# Create new columns based on median comparison
cat_data['big_employees'] = np.where(cat_data['number_of_employee'] > median_employee, 1, 0)
cat_data['big_assets'] = np.where(cat_data['assets_under_management'] > median_assets, 1, 0)

cat_data

,bank,number_of_employee,assets_under_management,number_of_branches,age,has_stores,founded_before_2000,web_traffic,big_employees,big_assets
0,abanca,5946,72148000000,690,13,1,0,1.357941e+06,0,0
1,santander,212764,1117000000000,8518,167,1,1,9.762759e+05,1,1
2,banc sabadell,19316,253000000000,1594,143,1,1,7.346442e+05,1,1
3,bankinter,6138,83300000000,523,59,1,1,2.755435e+05,1,0
4,bbva,121486,775000000000,1800,167,1,1,9.920828e+05,1,1
5,caixabank,44863,607167000000,3922,120,1,1,NaN,1,1
6,evobanco,200,7350000000,80,12,1,0,1.696284e+05,0,0
7,imagin,118,607167000000,0,8,0,0,NaN,0,1
8,ing,60778,976000000000,1660,33,1,1,7.016930e+05,1,1
9,kutxabank,5343,25000000000,737,12,1,0,NaN,0,0


In [25]:
data = pd.read_csv('data/regression_clean_data.csv')
data.head()

,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.0,150.0,0.0,1.0
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.0,150.0,0.0,1.0
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.0,150.0,0.0,2.0
3,abanca,2022-03-01,34546.319021,0.061503,6.75,4.0,150.0,0.0,2.0
4,abanca,2022-04-01,40508.288420,0.050222,6.75,4.0,300.0,0.0,2.0


In [26]:
# left join ['has_stores', 'founded_before_2000', 'big_employees', 'big_assets'] to data
data = data.merge(cat_data[['bank', 'has_stores', 'founded_before_2000', 'big_employees', 'big_assets']], on='bank', how='left')
data


,bank,date,web_traffic,cross_visitation,search_interest,rank,incentive,apr,mentions,has_stores,founded_before_2000,big_employees,big_assets
0,abanca,2021-12-01,15202.983835,0.077994,7.75,4.000000,150.0,0.0,1.0,1,0,0,0
1,abanca,2022-01-01,42183.913207,0.062500,7.80,4.000000,150.0,0.0,1.0,1,0,0,0
2,abanca,2022-02-01,34498.153115,0.052288,7.25,4.000000,150.0,0.0,2.0,1,0,0,0
3,abanca,2022-03-01,34546.319021,0.061503,6.75,4.000000,150.0,0.0,2.0,1,0,0,0
4,abanca,2022-04-01,40508.288420,0.050222,6.75,4.000000,300.0,0.0,2.0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,santander,2023-08-01,34290.160425,0.118914,50.50,3.500000,150.0,0.0,1.0,1,1,1,1
271,santander,2023-09-01,40388.737821,0.125496,56.25,3.500000,400.0,0.0,3.0,1,1,1,1
272,santander,2023-10-01,36636.706633,0.126347,58.60,3.500000,400.0,0.0,4.0,1,1,1,1
273,santander,2023-11-01,35387.630139,0.121212,59.75,1.500000,400.0,0.0,3.0,1,1,1,1


In [27]:
# log scale web traffic
data['log_web_traffic'] = np.log(data['web_traffic'])

### Panel Regressions

with lag

In [ ]:
fixed_effects = data.copy()

fixed_effects.date = pd.to_datetime(fixed_effects.date)

fixed_effects['web_traffic_lag'] = fixed_effects['web_traffic'].shift(1)

# Convert the DataFrame to a panel DataFrame
fixed_effects.set_index(['bank', 'date'], inplace=True)

In [38]:
# Define the dependent and independent variables
y = fixed_effects['log_web_traffic']
X = fixed_effects[['cross_visitation', 'search_interest', 'rank', 'incentive', 'apr', 'mentions', 'web_traffic_lag']]
X = sm.add_constant(X)

# Create and fit the fixed effects model
model = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
fe_results = model.fit()

# Display the summary
print(fe_results)

                          PanelOLS Estimation Summary                           
Dep. Variable:        log_web_traffic   R-squared:                        0.2038
Estimator:                   PanelOLS   R-squared (Between):              0.5443
No. Observations:                 274   R-squared (Within):               0.2038
Date:                Thu, Jun 20 2024   R-squared (Overall):              0.4740
Time:                        14:49:36   Log-likelihood                   -113.19
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      9.3629
Entities:                          11   P-value                           0.0000
Avg Obs:                       24.909   Distribution:                   F(7,256)
Min Obs:                       24.000                                           
Max Obs:                       25.000   F-statistic (robust):             9.3629
                            

/Users/andrewbennett/Library/Python/3.9/lib/python/site-packages/linearmodels/panel/model.py:1219: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


### Multicollinearity check
VIF = 1: No multicollinearity.
1 < VIF < 5: Moderate multicollinearity.
VIF >= 5: High multicollinearity. Variables with VIFs above 5 require further investigation and potentially should be removed or combined with other variables.

In [58]:
X.fillna(0, inplace=True)

vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif)

           Variable        VIF
0             const  23.765031
1  cross_visitation   1.977825
2   search_interest   1.816889
3              rank   1.340604
4         incentive   1.457948
5               apr   1.298543
6          mentions   1.157477
7   web_traffic_lag   1.939567


### Pooled Data Regression

dummy cols casted

In [ ]:
pooled_data = pd.get_dummies(data, columns=['bank'], drop_first=True)


# cast the dummy columns to int 
pooled_data[[ 'bank_banc sabadell', 'bank_bankinter',
       'bank_bbva', 'bank_evobanco', 'bank_ing', 'bank_myinvestor', 'bank_n26',
       'bank_openbank', 'bank_revolut', 'bank_santander','has_stores','founded_before_2000', 'big_employees', 'big_assets']] = pooled_data[[ 'bank_banc sabadell', 'bank_bankinter',
       'bank_bbva', 'bank_evobanco', 'bank_ing', 'bank_myinvestor', 'bank_n26',
       'bank_openbank', 'bank_revolut', 'bank_santander','has_stores','founded_before_2000', 'big_employees', 'big_assets']].astype(int)

big banks only

In [55]:

big_v_small_cols =  ['has_stores','founded_before_2000', 'big_employees', 'big_assets']
interaction_terms = []

for col1 in big_v_small_cols: 
       reg = []
       for col2 in ['cross_visitation', 'search_interest', 'rank',
                     'incentive', 'apr', 'mentions', 'web_traffic_lag']:
              pooled_data[col1 + ' x ' + col2] = pooled_data[col1] * pooled_data[col2]
              reg.append(col1 + ' x ' + col2)
       interaction_terms.append(reg)

pooled_data = pooled_data.fillna(0)

In [59]:

for index in range(0, len(interaction_terms)): 
    # Define the dependent and independent variables
    y = pooled_data['log_web_traffic']

    X = pooled_data[interaction_terms[index]]

    X = sm.add_constant(X)

    # Create and fit the fixed effects model
    model = sm.OLS(y, X).fit()
    pooled_regression_summary = model.summary()
    print(pooled_regression_summary)


                            OLS Regression Results                            
Dep. Variable:        log_web_traffic   R-squared:                       0.687
Model:                            OLS   Adj. R-squared:                  0.679
Method:                 Least Squares   F-statistic:                     83.83
Date:                Thu, 20 Jun 2024   Prob (F-statistic):           1.02e-63
Time:                        15:14:46   Log-Likelihood:                -200.50
No. Observations:                 275   AIC:                             417.0
Df Residuals:                     267   BIC:                             445.9
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                    coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
const         

small banks now

In [60]:

big_v_small_cols =  ['has_stores','founded_before_2000', 'big_employees', 'big_assets']
big_v_small_cols_invert =  ['no_stores','founded_after_2000', 'small_employees', 'small_assets']
pooled_data_inverted = pooled_data.copy()
interaction_terms_invert = []

index = 0
for col1 in big_v_small_cols: 
       reg = []
       # invert 0 to 1 and 1 to 0 in col1 
       pooled_data_inverted[col1] = pooled_data_inverted[col1].apply(lambda x: 1 if x == 0 else 0)
       for col2 in ['cross_visitation', 'search_interest', 'rank',
                     'incentive', 'apr', 'mentions', 'web_traffic_lag']:
              pooled_data_inverted[big_v_small_cols_invert[index] + ' x ' + col2] = pooled_data_inverted[col1] * pooled_data_inverted[col2]
              reg.append(big_v_small_cols_invert[index] + ' x ' + col2)
       interaction_terms_invert.append(reg)
       index += 1

pooled_data_inverted = pooled_data_inverted.fillna(0)

In [62]:

for index in range(0, len(interaction_terms_invert)): 
    # Define the dependent and independent variables
    y = pooled_data_inverted['log_web_traffic']

    X = pooled_data_inverted[interaction_terms_invert[index]]

    X = sm.add_constant(X)

    # Create and fit the fixed effects model
    model = sm.OLS(y, X).fit()
    pooled_regression_summary = model.summary()
    print(pooled_regression_summary)


                            OLS Regression Results                            
Dep. Variable:        log_web_traffic   R-squared:                       0.281
Model:                            OLS   Adj. R-squared:                  0.265
Method:                 Least Squares   F-statistic:                     17.47
Date:                Fri, 21 Jun 2024   Prob (F-statistic):           4.63e-17
Time:                        10:25:14   Log-Likelihood:                -314.94
No. Observations:                 275   AIC:                             643.9
Df Residuals:                     268   BIC:                             669.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

cross visitation search interest all significant for big asset banks

search interest and rank significant for large employee banks 

cross visitation incentive and apr are significant for old banks 

cross visitation incentive apr significant for brick and mortar banks


cross visitation and rank significant for small asset banks

rank and apr significant for small employee banks 

cross visitation and rank significant for new banks 

cross visitation search interest rank significant for neo banks

incentive and apr are more significant for older banks with a brick and mortar presence while newer banks with no physical presence are more affected by search interest and rank. 
This is interesting because is means the newer banks are relying on digital advertising and search engine optimization to drive traffic to their websites.

rank and cross visitation are significant across most model specifications which means that all of the banks still rely on digital advertising to drive traffic to their websites.

In [ ]:
X.fillna(0, inplace=True)

vif = pd.DataFrame()
vif["Variable"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif)